In [3]:
import tensorflow_datasets as tfds
# import tensorflow as tf

dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
dataset_size = info.splits["train"].num_examples # 3670
class_names = info.features["label"].names # ["dandelion", "daisy", ...]
n_classes = info.features["label"].num_classes # 5

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset tf_flowers downloaded and prepared to /home/vu/tensorflow_datasets/tf_flowers/3.0.0. Subsequent calls will reuse this data.


In [4]:
print(dataset_size,class_names, n_classes)

3670 ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses'] 5


In [8]:
# splits = tfds.Split.TRAIN.subsplit([10, 15, 75])
# (train_set, valid_set, test_set), metadata = tfds.load("tf_flowers", split=list(splits), as_supervised=True)

(train_set, valid_set, test_set), metadata = tfds.load(
'tf_flowers',
split=['train[:75%]', 'train[75%:90%]', 'train[90%:]'],
with_info=True,
as_supervised=True,)

In [9]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [11]:
from tensorflow import keras

batch_size = 32
train_set = train_set.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1)

In [12]:
base_model = keras.applications.xception.Xception(weights="imagenet",include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

83689472/83683744 [==============================] - 55s 1us/step


In [13]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,metrics=["accuracy"])
history = model.fit(train_set, epochs=5, validation_data=valid_set)

Epoch 1/5
86/86 [==============================] - 490s 6s/step - loss: 0.7532 - accuracy: 0.7674 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
17/18 [===========================>..] - ETA: 5s - loss: 0.3684 - accuracy: 0.8879 

In [ ]:
for layer in base_model.layers:
layer.trainable = True
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,metrics=["accuracy"])
history = model.fit(train_set, epochs=5, validation_data=valid_set)